# Reference: https://medium.com/@guildbilla/steel-defect-detection-image-segmentation-using-keras-dae8b4f986f0

# ***Improve***

# re-train

1. less val data? 

2. more augmentation (albumentations?)

3. more batch size ?

4. higher epoch times

# inference

5. threshold change ?

# other

6. Pseudo Labels?

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/final_project')

Mounted at /content/gdrive


# Import

In [2]:
!pip install segmentation_models

     |████████████████████████████████| 51kB 5.1MB/s 


In [3]:
!pip install q tensorflow==2.1
!pip install q keras==2.3.1

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 3.9MB 39.4MB/s 
     |████████████████████████████████| 450kB 39.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=e0c4e8e4aab3bc27621f00a3d789942406d351e1b5c45cd2c7c27af3d492739a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: 

In [24]:
# Import需要的套件
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time
from PIL import Image
from tensorflow import keras
from keras import layers

import keras
import tensorflow as tf
from tensorflow.python.keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import recall_score
from random import random
from random import seed

# https://github.com/qubvel/segmentation_models
import segmentation_models
print(segmentation_models.__version__)

import segmentation_models as sm
from segmentation_models import Unet
from segmentation_models import get_preprocessing

from tensorflow.keras.utils import plot_model

from time import time
from datetime import datetime
from keras import backend as K

from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator

import albumentations as A
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import Sequence

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

1.0.1


# Read Data & Make dataframe

In [5]:
train_path='./train_images/'
train_image_names=os.listdir(train_path)
print(train_image_names)
print(len(train_image_names))

['ea4f8eb1d.jpg', 'e9f4baa36.jpg', 'ea476b31f.jpg', 'ebb817ec9.jpg', 'e8a40861a.jpg', 'eb4c89edf.jpg', 'e9ab2fa1c.jpg', 'ec58aa10d.jpg', 'eadff74bc.jpg', 'e89dfa2ad.jpg', 'eb5a90623.jpg', 'ec4b5010d.jpg', 'ea0cac9af.jpg', 'eb2b76683.jpg', 'ead7d8e70.jpg', 'ea23dae53.jpg', 'ec3fe85d6.jpg', 'e9e95609c.jpg', 'eb4225311.jpg', 'e9ccfe4b5.jpg', 'eb5677124.jpg', 'ebe4ba4f4.jpg', 'e8aacce9c.jpg', 'eae952e95.jpg', 'e9cff204e.jpg', 'e9ccf81cf.jpg', 'ec37ebae6.jpg', 'eba3998d7.jpg', 'ebfa0cc57.jpg', 'eb8aa849b.jpg', 'e8f6b3f16.jpg', 'e9eb5a464.jpg', 'ec0e3a1c2.jpg', 'eabd4acaa.jpg', 'e9fa75516.jpg', 'ea48b3962.jpg', 'ebeaefa1b.jpg', 'e9e65d5f1.jpg', 'e98ffbe3d.jpg', 'e9d70c726.jpg', 'eaf7443a7.jpg', 'eb0c5c725.jpg', 'ec1ab47d4.jpg', 'ebffb5d31.jpg', 'eb5aec756.jpg', 'e9e04715b.jpg', 'ead253ed7.jpg', 'eb10d7ae4.jpg', 'eb2008abd.jpg', 'ec2ce528a.jpg', 'ead245f1f.jpg', 'ec0c4dd47.jpg', 'ebb8247c6.jpg', 'e9ed21340.jpg', 'eaa65effb.jpg', 'e9309d54e.jpg', 'eb2b64947.jpg', 'e943bbc5b.jpg', 'ebb794625.jp

In [6]:
def get_label(path):
	df = pd.read_csv(path)
	Ydict = {}
	for image_id in df.ImageId.unique():
		rle = [
			df.loc[ (df['ImageId']==image_id) & (df['ClassId'] == 1) , 'EncodedPixels'].values,
			df.loc[ (df['ImageId']==image_id) & (df['ClassId'] == 2) , 'EncodedPixels'].values,
			df.loc[ (df['ImageId']==image_id) & (df['ClassId'] == 3) , 'EncodedPixels'].values,
			df.loc[ (df['ImageId']==image_id) & (df['ClassId'] == 4) , 'EncodedPixels'].values
		]
		for classID in range(4):
			if len(rle[classID]) == 0:
				rle[classID] = ''
			else:
				rle[classID] = rle[classID][0]

		Ydict[image_id ] =  rle
	del df
	return Ydict


In [7]:
Ydict=get_label('./train.csv')

In [8]:
imageid=[]
for i in train_image_names:
    imageid.append(i)
    imageid.append(i)
    imageid.append(i)
    imageid.append(i)
print(imageid)
print(len(imageid))

classid=[1,2,3,4]*12568
print(classid)
print(len(classid))

encodedpixel=[]
for i in train_image_names:
    if i in Ydict:
        encodedpixel.append(Ydict[i][0])
        encodedpixel.append(Ydict[i][1])
        encodedpixel.append(Ydict[i][2])
        encodedpixel.append(Ydict[i][3])
    else:
        encodedpixel.append('')
        encodedpixel.append('')
        encodedpixel.append('')
        encodedpixel.append('')
#print(encodedpixel)
#print(len(encodedpixel))

['ea4f8eb1d.jpg', 'ea4f8eb1d.jpg', 'ea4f8eb1d.jpg', 'ea4f8eb1d.jpg', 'e9f4baa36.jpg', 'e9f4baa36.jpg', 'e9f4baa36.jpg', 'e9f4baa36.jpg', 'ea476b31f.jpg', 'ea476b31f.jpg', 'ea476b31f.jpg', 'ea476b31f.jpg', 'ebb817ec9.jpg', 'ebb817ec9.jpg', 'ebb817ec9.jpg', 'ebb817ec9.jpg', 'e8a40861a.jpg', 'e8a40861a.jpg', 'e8a40861a.jpg', 'e8a40861a.jpg', 'eb4c89edf.jpg', 'eb4c89edf.jpg', 'eb4c89edf.jpg', 'eb4c89edf.jpg', 'e9ab2fa1c.jpg', 'e9ab2fa1c.jpg', 'e9ab2fa1c.jpg', 'e9ab2fa1c.jpg', 'ec58aa10d.jpg', 'ec58aa10d.jpg', 'ec58aa10d.jpg', 'ec58aa10d.jpg', 'eadff74bc.jpg', 'eadff74bc.jpg', 'eadff74bc.jpg', 'eadff74bc.jpg', 'e89dfa2ad.jpg', 'e89dfa2ad.jpg', 'e89dfa2ad.jpg', 'e89dfa2ad.jpg', 'eb5a90623.jpg', 'eb5a90623.jpg', 'eb5a90623.jpg', 'eb5a90623.jpg', 'ec4b5010d.jpg', 'ec4b5010d.jpg', 'ec4b5010d.jpg', 'ec4b5010d.jpg', 'ea0cac9af.jpg', 'ea0cac9af.jpg', 'ea0cac9af.jpg', 'ea0cac9af.jpg', 'eb2b76683.jpg', 'eb2b76683.jpg', 'eb2b76683.jpg', 'eb2b76683.jpg', 'ead7d8e70.jpg', 'ead7d8e70.jpg', 'ead7d8e70.jp

In [9]:
data={'ImageId':imageid,'ClassId':classid,'EncodedPixels':encodedpixel}
train_df=pd.DataFrame(data,columns=['ImageId','ClassId','EncodedPixels'])
train_df

,ImageId,ClassId,EncodedPixels
0,ea4f8eb1d.jpg,1,
1,ea4f8eb1d.jpg,2,
2,ea4f8eb1d.jpg,3,224 33 416 97 608 161 831 194 1087 194 1343 19...
3,ea4f8eb1d.jpg,4,
4,e9f4baa36.jpg,1,139441 22 139675 6 139688 37 139925 59 140168 ...
...,...,...,...
50267,0984c380f.jpg,4,
50268,09d4ee8a5.jpg,1,
50269,09d4ee8a5.jpg,2,
50270,09d4ee8a5.jpg,3,74756 253 75012 253 75268 253 75524 253 75780 ...


In [10]:
train_data=pd.pivot_table(train_df,values='EncodedPixels',index='ImageId',columns='ClassId',aggfunc=np.sum).astype(str)
train_data=train_data.reset_index()
train_data.columns=['ImageId','Defect_1','Defect_2','Defect_3','Defect_4']
train_data

,ImageId,Defect_1,Defect_2,Defect_3,Defect_4
0,0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,,,
1,00031f466.jpg,,,,
2,000418bfc.jpg,,,,
3,000789191.jpg,,,,
4,0007a71bf.jpg,,,18661 28 18863 82 19091 110 19347 110 19603 11...,
...,...,...,...,...,...
12563,fff0295e1.jpg,,,,
12564,fff02e9c5.jpg,,,207523 3 207777 9 208030 15 208283 22 208537 2...,
12565,fffe98443.jpg,,,105929 5 106177 14 106424 24 106672 33 106923 ...,
12566,ffff4eaa8.jpg,,,16899 7 17155 20 17411 34 17667 47 17923 60 18...,


modify dataframe

In [11]:
Imageid=train_data.loc[:,'ImageId']
tmp=list(Imageid)
hasdefect=[]
hasdefect_1=[]
hasdefect_2=[]
hasdefect_3=[]
hasdefect_4=[]
stratify=[]
for i in tmp:
    if i in Ydict:
        j=Ydict[i]
        c=0 #class
        if len(j[0])>0:
            hasdefect_1.append(1)
            c=1
        else:
            hasdefect_1.append(0)
        if len(j[1])>0:
            hasdefect_2.append(1)
            c=2
        else:
            hasdefect_2.append(0)
        if len(j[2])>0:
            hasdefect_3.append(1)
            c=3
        else:
            hasdefect_3.append(0)
        if len(j[3])>0:
            hasdefect_4.append(1)
            c=4
        else:
            hasdefect_4.append(0)

        if c==0:
            hasdefect.append(0)
            stratify.append(0)
        else:
            hasdefect.append(1)
            stratify.append(c)
    else:
        hasdefect.append(0)
        hasdefect_1.append(0)
        hasdefect_2.append(0)
        hasdefect_3.append(0)
        hasdefect_4.append(0)
        stratify.append(0)

print(len(hasdefect))
print(len(hasdefect_1))
print(len(hasdefect_2))
print(len(hasdefect_3))
print(len(hasdefect_4))
print(len(stratify))

12568
12568
12568
12568
12568
12568


In [12]:
d={'ImageId':tmp,'hasDefect':hasdefect,'hasDefect_1':hasdefect_1,'hasDefect_2':hasdefect_2,'hasDefect_3':hasdefect_3,'hasDefect_4':hasdefect_4,'stratify':stratify}
tobemerge=pd.DataFrame(d,columns=['ImageId','hasDefect','hasDefect_1','hasDefect_2','hasDefect_3','hasDefect_4','stratify'])
train_data=pd.merge(train_data,tobemerge)
train_data.head()

,ImageId,Defect_1,Defect_2,Defect_3,Defect_4,hasDefect,hasDefect_1,hasDefect_2,hasDefect_3,hasDefect_4,stratify
0,0002cc93b.jpg,29102 12 29346 24 29602 24 29858 24 30114 24 3...,,,,1,1,0,0,0,1
1,00031f466.jpg,,,,,0,0,0,0,0,0
2,000418bfc.jpg,,,,,0,0,0,0,0,0
3,000789191.jpg,,,,,0,0,0,0,0,0
4,0007a71bf.jpg,,,18661 28 18863 82 19091 110 19347 110 19603 11...,,1,0,0,1,0,3


# Train / Val split

In [13]:
from sklearn.model_selection import train_test_split
X=train_data.copy()

data_size=train_data.shape[0]
train_size=10054
val_size=data_size-train_size

X_train , X_val = train_test_split(X, test_size=val_size, stratify=X['stratify'], random_state=42)

print(X_train.shape, X_val.shape)

(10054, 11) (2514, 11)


# Distribution

## X_train

class1: 716

class2: 194

class3: 4115

class4: 641

has label: 5333

no label: 4721

## X_val

class1: 181

class2: 53

class3: 1035

class4: 160

has label: 1333

no label: 1181

# Run-length coding

rle=run length encoding ,string format

mask=256 * 1600 np.array from rle

In [14]:
def rle2mask(rle, height=256, width=1600, fill_value=1):
	mask = np.zeros((height,width), np.float32)
	if rle != '':
		mask=mask.reshape(-1)
		r = [int(r) for r in rle.split(' ')]
		r = np.array(r).reshape(-1, 2)
		for start,length in r:
			#start = start-1  #???? 0 or 1 index ???        #0
			mask[start:(start + length)] = fill_value
		mask=mask.reshape(width, height).T
	return mask


In [15]:
def mask2rle(mask):
#possible bug for here
	m = mask.T.flatten()
	if m.sum()==0:
		rle=''
	else:
		m   = np.concatenate([[0], m, [0]])
		run = np.where(m[1:] != m[:-1])[0] + 1
		run[1::2] -= run[::2]
		rle = ' '.join(str(r) for r in run)
	return rle


# "Area" as new feature, Area for class n for image = rle2mask.sum() on class n

# Area thresholds , 最後產生answer時用到

defect       /Min     / Max     

defect_1    / 500   /   15500 



defect_2    / 700   /   10000 



defect_3   / 1100   /  160000 




defect_4   / 2800   / 127000 



# Models



1.   Binary Classifier:  

        --> train with all images
2.   Multi-class Classifier(output dim=4):  

        --> train with images have defects (if no defect image involved , it will error)
3.   Segmentation Predicter * 4 : 

        --> train with each defect images separately



In [16]:
train_segmentation = True
train_classification_binary = True
train_classification_multi = True
epochs = 30

In [17]:
# Metrics
# For image segmentation
# COMPETITION METRIC
# https://www.kaggle.com/xhlulu/severstal-simple-keras-u-net-boilerplate
def dice_coef(y_true, y_pred, smooth=K.epsilon()):
    '''
    This function returns dice coefficient of similarity between y_true and y_pred
    Dice coefficient is also referred to as F1_score, but we will use this name for image segmentation models
    For example, 
    let an instance on y_true and y_pred be [[1,1],[0,1]] and [[1,0],[0,1]]
    this metric first converts the above into [1,1,0,1] abd [1,0,0,1],
    then intersection is calculated as 1*1 + 1*0 + 0*1 + 1*1 = 2 and sum(y_true)+sum(y_pred)= 3+2 = 5
    this returns the value (2.* 2 + 10e-7)/(3 + 2 + 10e-7) ~ 0.8    
    '''
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Custom metrics, https://stackoverflow.com/questions/59196793/why-are-my-metrics-of-my-cnn-not-changing-with-each-epoch
# For clasification
def recall_m(y_true, y_pred):
    '''
    This function returns recall_score between y_true and y_pred
    This function is ported as a metric to the Neural Network Models
    Keras backend is used to take care of batch type training, the metric takes in a batch of y_pred and corresponding y_pred 
    as input and returns recall score of the batch
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) # calculates number of true positives
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))      # calculates number of actual positives
    recall = true_positives / (possible_positives + K.epsilon())   # K.epsilon takes care of non-zero divisions
    return recall

def precision_m(y_true, y_pred):
    '''
    This function returns precison_score between y_true and y_pred
    This function is ported as a metric to the Neural Network Models
    Keras backend is used to take care of batch type training, the metric takes in a batch of y_pred and corresponding y_pred 
    as input and returns prediction score of the batch
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))  # calculates number of true positives
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))      # calculates number of predicted positives   
    precision = true_positives /(predicted_positives + K.epsilon()) # K.epsilon takes care of non-zero divisions
    return precision
    
def f1_score_m(y_true, y_pred):
    '''
    This function returns f1_score between y_true and y_pred
    This 
    This function is ported as a metric to the Neural Network Models
    Keras backend is used to take care of batch type training, the metric takes in a batch of y_pred and corresponding y_pred 
    as input and returns f1 score of the batch
    '''
    precision = precision_m(y_true, y_pred)  # calls precision metric and takes the score of precision of the batch
    recall = recall_m(y_true, y_pred)        # calls recall metric and takes the score of precision of the batch
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

dependencies = {
    'recall_m':recall_m,
    'precision_m':precision_m,
    'dice_coef':dice_coef,
    'f1_score_m':f1_score_m,
    'dice_loss':sm.losses.dice_loss
}

#  Binary Classification

In [18]:
#Binary Classifier
X_train_binary=X_train[['ImageId','hasDefect']]
X_val_binary=X_val[['ImageId','hasDefect']]

print(X_train_binary.shape, X_val_binary.shape)

(10054, 2) (2514, 2)


In [19]:
class AugmentDataGenerator(Sequence):
    def __init__(self, datagen, augment=None):
        self.datagen = datagen
        if augment is None:
            self.augment = A.Compose([])
        else:
            self.augment = augment

    def __len__(self):
        return len(self.datagen)

    def __getitem__(self, x):
        images, *rest = self.datagen[x]
        augmented = []
        for image in images:
            image = self.augment(image=image)['image']
            augmented.append(image)
        return (np.array(augmented), *rest)

In [20]:
#Data Preprocessing
#normalize / data augmentation / image to tensor 

train_DataGenerator_1 = keras.preprocessing.image.ImageDataGenerator(rescale=1./255. , shear_range=0, zoom_range=0.05, rotation_range=0, 
                                                                   width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, vertical_flip=True)

test_DataGenerator_1 = keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

train_generator_1 = train_DataGenerator_1.flow_from_dataframe(
    dataframe= X_train_binary.astype(str),
    directory= train_path,
    x_col= "ImageId",
    y_col= "hasDefect",
    target_size= (256,512),
    batch_size= 16,
    class_mode= "binary"
)

validation_generator_1 = test_DataGenerator_1.flow_from_dataframe(
    dataframe= X_val_binary.astype(str),
    directory= train_path,
    x_col= "ImageId",
    y_col= "hasDefect",
    target_size= (256,512),
    batch_size= 16,
    class_mode= "binary"
)

Found 10054 validated image filenames belonging to 2 classes.
Found 2514 validated image filenames belonging to 2 classes.


In [21]:
train_generator= AugmentDataGenerator(train_generator_1, A.Compose([
    A.RandomCrop(256,480),
    A.Resize(256,512),
    A.RandomBrightnessContrast(p=0.2),
]))

validation_generator= AugmentDataGenerator(validation_generator_1,augment=None)

In [22]:
#Binary model definition
#Binary class model by keras xception (pre-train model)  for classification
#https://keras.io/api/applications/


base_model=keras.applications.xception.Xception(include_top= False, input_shape=(256,512,3))

#add a global spatial average pooling layer
x= base_model.output
x= layers.GlobalAveragePooling2D()(x)

#add a fully connected layer
x= layers.Dense(1024, activation='relu')(x)
x= layers.BatchNormalization()(x)
x= layers.Dropout(0.3)(x)

x= layers.Dense(512, activation='relu')(x)
x= layers.BatchNormalization()(x)
x= layers.Dropout(0.3)(x)

x= layers.Dense(64, activation='relu')(x)

#prediction layer
predictions = layers.Dense(1,activation='sigmoid')(x)

#model we will train
model=keras.Model(inputs= base_model.input , outputs= predictions) #parameters: 23523497

83689472/83683744 [==============================] - 3s 0us/step


In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 512, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 255, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 255, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 255, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [ ]:
#binary classification model training

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc',f1_score_m,precision_m,recall_m])
binary_model_path='./binary_model_0108.hdf5' # where model saved
if train_classification_binary==True:

    mc = ModelCheckpoint(filepath=binary_model_path, monitor='val_f1_score_m', mode='max', verbose=1, save_best_only=True)
    model.fit(train_generator, validation_data = validation_generator, epochs = epochs, verbose=1, callbacks = [mc])


Epoch 1/30
 89/629 [===>..........................] - ETA: 1:21:22 - loss: 0.7368 - acc: 0.6678 - f1_score_m: 0.6898 - precision_m: 0.6883 - recall_m: 0.7313

# Binary Classification Evaluate

In [25]:
# Loading best model trained on binary classification
model= load_model('./binary_model_1204.hdf5', custom_objects=dependencies)

In [26]:
# During evaluation image augmentations are not to be included thus the DataGenerators are redefined.

train_generator = test_DataGenerator_1.flow_from_dataframe(dataframe=X_train_binary.astype(str),
                                                           directory=train_path,
                                                           x_col="ImageId",
                                                           y_col="hasDefect",
                                                           target_size=(256,512),
                                                           batch_size=16,
                                                           class_mode='binary',
                                                           shuffle=False)

validation_generator = test_DataGenerator_1.flow_from_dataframe(dataframe=X_val_binary.astype(str),
                                                                directory=train_path,
                                                                x_col="ImageId",
                                                                y_col="hasDefect",
                                                                target_size=(256,512),
                                                                batch_size=16,
                                                                class_mode='binary',
                                                                shuffle=False)

Found 9048 validated image filenames belonging to 2 classes.
Found 3520 validated image filenames belonging to 2 classes.


In [27]:
train_evaluate = model.evaluate(train_generator,verbose=1)
print('Train set evaluation score:')
pd.DataFrame(train_evaluate,columns = [' '], index=['binary_crossentropy','acc','f1_score_m','precision_m','recall_m'])

566/566 [==============================] - 2201s 4s/step
Train set evaluation score:


,
binary_crossentropy,0.138676
acc,0.935566
f1_score_m,0.935465
precision_m,0.928266
recall_m,0.951102


In [28]:
val_evaluate = model.evaluate(validation_generator,verbose=1)
print('Validation set evaluation score:')
pd.DataFrame(val_evaluate,columns = [' '], index=['binary_crossentropy','acc','f1_score_m','precision_m','recall_m'])

220/220 [==============================] - 849s 4s/step
Validation set evaluation score:


,
binary_crossentropy,0.366309
acc,0.926989
f1_score_m,0.926840
precision_m,0.922778
recall_m,0.940176


# Multi-label classification

Data preparation

In [18]:
#train with image have defect only
 
X_train_multi = X_train[['ImageId','hasDefect_1','hasDefect_2','hasDefect_3','hasDefect_4']][X_train['hasDefect']==1]
X_val_multi = X_val[['ImageId','hasDefect_1','hasDefect_2','hasDefect_3','hasDefect_4']][X_val['hasDefect']==1]

print(X_train.shape, X_val.shape)
print(X_train_multi.shape, X_val_multi.shape)

(9048, 11) (3520, 11)
(4799, 5) (1867, 5)


In [19]:
# https://keras.io/preprocessing/image/
# https://stackoverflow.com/questions/52754492/write-custom-data-generator-for-keras

# DataGenerator for the multi label classification model with image augmentations
train_DataGenerator_2 = keras.preprocessing.image.ImageDataGenerator(rescale=1./255., shear_range=0, zoom_range=0.05, rotation_range=0,
                           width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, vertical_flip=True)

test_DataGenerator_2 = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


train_generator_2 = train_DataGenerator_2.flow_from_dataframe(
        dataframe=X_train_multi.astype(str),
        directory=train_path,
        x_col="ImageId",
        y_col=["hasDefect_1","hasDefect_2","hasDefect_3","hasDefect_4"],
        target_size=(256,512),
        batch_size=16,
        class_mode="raw") #'other'?

validation_generator_2 = test_DataGenerator_2.flow_from_dataframe(
        dataframe=X_val_multi.astype(str),
        directory=train_path,
        x_col="ImageId",
        y_col=["hasDefect_1","hasDefect_2","hasDefect_3","hasDefect_4"],
        target_size=(256,512),
        batch_size=16,
        class_mode="raw")

Found 4799 validated image filenames.
Found 1867 validated image filenames.


In [ ]:
train_generator = AugmentDataGenerator(train_generator_2, A.Compose([
    A.RandomCrop(256,480),
    A.Resize(256,512),
    A.RandomBrightnessContrast(p=0.2),
]))

validation_generator = AugmentDataGenerator(validation_generator_2, augment= None)

Multi-Label Classification Model Definition

In [20]:
# Using a pretrained model from keras for classification: 
# Selecting Xception pretrained model
# https://keras.io/applications/

base_model = keras.applications.xception.Xception(include_top = False, input_shape = (256,512,3))

# add a global spatial average pooling layer
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)

# let's add fully-connected layers
x = layers.Dense(1024, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(64, activation='relu')(x)

# and the prediction layer
predictions = layers.Dense(4, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

83689472/83683744 [==============================] - 2s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 512, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 255, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 255, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 255, 32) 0           block1_conv1_bn[0][0]            
___________________________

Multi-label Classification Model Training 

In [21]:

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc',f1_score_m,precision_m,recall_m])
multi_class_model_path='./multi_class_model_0105.h5'
if train_classification_multi==True:

    mc = ModelCheckpoint(filepath = multi_class_model_path , monitor='val_f1_score_m', mode='max', verbose=1, save_best_only=True)
    model.fit(train_generator, validation_data = validation_generator, epochs = epochs, verbose=1, callbacks = [mc])


Epoch 1/30
300/300 [==============================] - 2753s 9s/step - loss: 0.3538 - acc: 0.8594 - f1_score_m: 0.7348 - precision_m: 0.7535 - recall_m: 0.7249 - val_loss: 0.1395 - val_acc: 0.8824 - val_f1_score_m: 0.7723 - val_precision_m: 0.8051 - val_recall_m: 0.7438

Epoch 00001: val_f1_score_m improved from -inf to 0.77233, saving model to ./multi_class_model_0105.h5
Epoch 2/30
300/300 [==============================] - 411s 1s/step - loss: 0.2738 - acc: 0.8898 - f1_score_m: 0.7885 - precision_m: 0.8072 - recall_m: 0.7745 - val_loss: 0.3560 - val_acc: 0.9100 - val_f1_score_m: 0.8108 - val_precision_m: 0.9162 - val_recall_m: 0.7321

Epoch 00002: val_f1_score_m improved from 0.77233 to 0.81081, saving model to ./multi_class_model_0105.h5
Epoch 3/30
300/300 [==============================] - 411s 1s/step - loss: 0.2351 - acc: 0.9005 - f1_score_m: 0.8085 - precision_m: 0.8289 - recall_m: 0.7925 - val_loss: 0.2264 - val_acc: 0.8897 - val_f1_score_m: 0.7866 - val_precision_m: 0.8166 - va

# Image segmentation

Data preparation

In [22]:
# Dividing the datasets w.r.t. Class Label (defect type)
train_data_1 = X_train[X_train['hasDefect_1']==1][['ImageId','Defect_1']]
train_data_2 = X_train[X_train['hasDefect_2']==1][['ImageId','Defect_2']]
train_data_3 = X_train[X_train['hasDefect_3']==1][['ImageId','Defect_3']]
train_data_4 = X_train[X_train['hasDefect_4']==1][['ImageId','Defect_4']]

val_data_1 = X_val[X_val['hasDefect_1']==1][['ImageId','Defect_1']]
val_data_2 = X_val[X_val['hasDefect_2']==1][['ImageId','Defect_2']]
val_data_3 = X_val[X_val['hasDefect_3']==1][['ImageId','Defect_3']]
val_data_4 = X_val[X_val['hasDefect_4']==1][['ImageId','Defect_4']]

train_data_1.columns = train_data_2.columns = train_data_3.columns = train_data_4.columns = ['ImageId','EncodedPixels']
val_data_1.columns = val_data_2.columns = val_data_3.columns = val_data_4.columns = ['ImageId','EncodedPixels']

print(X_train.shape, X_val.shape)
print(train_data_1.shape,val_data_1.shape)
print(train_data_2.shape,val_data_2.shape)
print(train_data_3.shape,val_data_3.shape)
print(train_data_4.shape,val_data_4.shape)

(9048, 11) (3520, 11)
(647, 2) (250, 2)
(173, 2) (74, 2)
(3698, 2) (1452, 2)
(577, 2) (224, 2)


In [23]:
# code reference, https://www.kaggle.com/cdeotte/keras-unet-with-eda
def rle2maskResize(rle):
    '''
    Generates masks for each image taking RLE as input
    Converts run length encoding to an image of shape defined uniform throughout segmentation models: 256x800
    Takes EncodedPixels as input, converts into 256x1600 mask and returns a resized mask image of size 256x800
    '''
    if (pd.isnull(rle))|(rle==''): # If the EncodedPixels string is empty an empty mask is returned
        return np.zeros((256,800) ,dtype=np.uint8)

    height= 256
    width = 1600
    mask= np.zeros( width*height ,dtype=np.uint8)

    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]-1 # The pixel array definition starts from 1 while array starts from 0
    lengths = array[1::2]  # The second element of EncodedPixels is the length denoting number of pixels in successive that are active (value = 1)
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1 # Making 
    
    return mask.reshape((height,width),order='F')[::,::2]

In [24]:
# https://www.kaggle.com/cdeotte/keras-unet-with-eda# https://www.kaggle.com/cdeotte/keras-unet-with-eda
# https://stackoverflow.com/questions/52754492/write-custom-data-generator-for-keras
# DataGenerator custom built for training segmentation models with random image augmentations
# 
class train_DataGenerator_3(keras.utils.Sequence): # with augmentation for training
    '''
    The DataGenerator takes a batch of ImageIds of batch size 8 and returns Image array to the model with its mask.
    With the help of ImageIds the DataGenerator locates the Image file in the path, the image is read and resized from
    256 x 1600 to 256x800.
    A set of random numbers are generated to generate random Image Augmentations.
    Shuffling is enabled during training to include variations in the sequence of images processed at each epoch.
    '''
    def __init__(self, df, batch_size = 8,  shuffle=True, 
                 preprocess=None, info={}):
        super().__init__()
        self.df = df
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.preprocess = preprocess
        self.info = info
        self.data_path = './train_images/'
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self, index): 
        X = np.empty((self.batch_size,256,800,3),dtype=np.float32)
        X1 = np.empty((self.batch_size,256,800,3),dtype=np.float32)

        y = np.empty((self.batch_size,256,800,1),dtype=np.int8)
        y1 = np.empty((self.batch_size,256,800,1),dtype=np.int8)

        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['ImageId'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = Image.open(self.data_path + f).resize((800,256))           
            y[i,:,:,0] = rle2maskResize(self.df['EncodedPixels'].iloc[indexes[i]])
        if self.preprocess!=None: X = self.preprocess(X)

        # generate some random image augmentations
        augment = random()
        if augment>0.35:
            in_gen1 = ImageDataGenerator()
            augment1 = random()
            augment2 = random()
            augment3 = random()
            augment4 = random()
            augment5 = random()
            augment6 = random()

            args = dict(tx = 0, ty = 0, zx = 1.0, zy= 1.0, flip_horizontal = False, flip_vertical = False)

            if augment1>0.5:
                args.update({'tx':50})

            if augment2>0.5:
                args.update({'ty':25})

            if augment3>0.5:
                args.update({'zx':0.9})

            if augment4>0.5:
                args.update({'zy':0.9})

            if augment5>0.5:
                args.update({'flip_horizontal' : True})

            if augment6>0.5:
                args.update({'flip_vertical' : True})

            for i,h in enumerate(X):
                X1[i] = in_gen1.apply_transform(h, transform_parameters = args)
            for i,g in enumerate(y):
                y1[i] = in_gen1.apply_transform(g, transform_parameters = args)
            return X1, y1
        else:
            return X, y

In [25]:
class test_DataGenerator_3(keras.utils.Sequence): # without augmentations for predictions
    '''
    The DataGenerator takes a batch of ImageIds of batch size 1 and returns Image array to the model with mask on validation
    dataset and without mask on test dataset.
    During Prediction and Evaluation stage Image augmentations are to not required. Thus this Train DataGenerator is modified 
    to create test Datagenerator
    With the help of ImageIds the DataGenerator locates the Image file in the path, the image is read and resized from
    256x1600 to 256x800.
    Shuffling is disabled during predictions to make sure each prediction belongs to its corresponding ImageId.
    '''
    def __init__(self, df, batch_size = 1, shuffle=False, 
                 preprocess=None, info={}):
        super().__init__()
        self.df = df
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.preprocess = preprocess
        self.info = info
        self.data_path = './train_images/'
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self, index): 
        X = np.empty((self.batch_size,256,800,3),dtype=np.float32)
        y = np.empty((self.batch_size,256,800,1),dtype=np.int8)

        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['ImageId'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = Image.open(self.data_path + f).resize((800,256))      
            y[i,:,:,0] = rle2maskResize(self.df['EncodedPixels'].iloc[indexes[i]])
        if self.preprocess!=None: X = self.preprocess(X)
        return X, y

Segmentation Model definition

In [26]:
# https://github.com/tensorflow/tpu/blob/master/models/official/efficientnet/preprocessing.py
# preprocesses image to input to the segmentation_model, generally image pixel value standardization
preprocess = get_preprocessing('efficientnetb1') 

# https://github.com/qubvel/segmentation_models
# segmentation using pretrained weights for faster convergence
model = Unet('efficientnetb1', classes=1, activation='sigmoid', encoder_weights='imagenet') 
model.summary()

27164672/27164032 [==============================] - 1s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 3 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 3 0           stem_bn[0][0]                    
___________________________

# Segmentation Model Training

Defect 1

In [32]:
if train_segmentation == True:
    # TRAIN AND VALIDATE MODEL
    # Defect 1
    model.compile(optimizer='adam', loss=sm.losses.dice_loss,metrics=[dice_coef])
    train_batches = train_DataGenerator_3(train_data_1,shuffle=True,preprocess=preprocess)    
    valid_batches = test_DataGenerator_3(val_data_1,preprocess=preprocess)
    
    # https://www.tensorflow.org/tensorboard/r2/scalars_and_keras
    #logdir = "/content/drive/My Drive/severstal_february/severstal_logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")+'_Defect_1_01_02_2020'
    #file_writer = tf.summary.FileWriter(logdir + "/metrics")
    #tensorboard = keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=0,write_images=True)
    
    # https://keras.io/callbacks/
    mc = ModelCheckpoint('./segment_model_defect1_0103.h5', monitor='val_dice_coef', mode='max', verbose=1, save_best_only=True)
    #model training
    history = model.fit_generator(train_batches, validation_data = valid_batches, epochs = epochs, verbose=1, callbacks = [mc])

Epoch 1/30
80/80 [==============================] - 403s 5s/step - loss: 0.8053 - dice_coef: 0.1947 - val_loss: 0.9985 - val_dice_coef: 0.0216

Epoch 00001: val_dice_coef improved from -inf to 0.02162, saving model to ./segment_model_defect1_0103.h5
Epoch 2/30
80/80 [==============================] - 102s 1s/step - loss: 0.4577 - dice_coef: 0.5423 - val_loss: 0.9568 - val_dice_coef: 0.2382

Epoch 00002: val_dice_coef improved from 0.02162 to 0.23816, saving model to ./segment_model_defect1_0103.h5
Epoch 3/30
80/80 [==============================] - 102s 1s/step - loss: 0.4111 - dice_coef: 0.5889 - val_loss: 0.8163 - val_dice_coef: 0.5497

Epoch 00003: val_dice_coef improved from 0.23816 to 0.54972, saving model to ./segment_model_defect1_0103.h5
Epoch 4/30
80/80 [==============================] - 101s 1s/step - loss: 0.4037 - dice_coef: 0.5963 - val_loss: 0.6405 - val_dice_coef: 0.6160

Epoch 00004: val_dice_coef improved from 0.54972 to 0.61601, saving model to ./segment_model_defect1

Defect 2 (lower)

In [27]:
if train_segmentation == True:
    # TRAIN AND VALIDATE MODEL
    # Defect 2
    model.compile(optimizer='adam', loss=sm.losses.dice_loss,metrics=[dice_coef])
    train_batches = train_DataGenerator_3(train_data_2,shuffle=True,preprocess=preprocess)    
    valid_batches = test_DataGenerator_3(val_data_2,preprocess=preprocess)
    
    # https://www.tensorflow.org/tensorboard/r2/scalars_and_keras
    #logdir = "/content/drive/My Drive/severstal_february/severstal_logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")+'_Defect_2_01_02_2020'
    #file_writer = tf.summary.FileWriter(logdir + "/metrics")
    #tensorboard = keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=0,write_images=True)
    
    # https://keras.io/callbacks/
    mc = ModelCheckpoint('./segment_model_defect2_0105.h5', monitor='val_dice_coef', mode='max', verbose=1, save_best_only=True)
    #model training
    history = model.fit_generator(train_batches, validation_data = valid_batches, epochs = epochs, verbose=1, callbacks = [mc])

Epoch 1/30
21/21 [==============================] - 69s 3s/step - loss: 0.9543 - dice_coef: 0.0457 - val_loss: 0.9804 - val_dice_coef: 0.0244

Epoch 00001: val_dice_coef improved from -inf to 0.02440, saving model to ./segment_model_defect2_0105.h5
Epoch 2/30
21/21 [==============================] - 27s 1s/step - loss: 0.8834 - dice_coef: 0.1166 - val_loss: 0.9972 - val_dice_coef: 0.0242

Epoch 00002: val_dice_coef did not improve from 0.02440
Epoch 3/30
21/21 [==============================] - 27s 1s/step - loss: 0.6823 - dice_coef: 0.3177 - val_loss: 0.9820 - val_dice_coef: 0.0202

Epoch 00003: val_dice_coef did not improve from 0.02440
Epoch 4/30
21/21 [==============================] - 27s 1s/step - loss: 0.5008 - dice_coef: 0.4992 - val_loss: 0.9818 - val_dice_coef: 0.0283

Epoch 00004: val_dice_coef improved from 0.02440 to 0.02829, saving model to ./segment_model_defect2_0105.h5
Epoch 5/30
21/21 [==============================] - 27s 1s/step - loss: 0.3849 - dice_coef: 0.6151 - 

Defect 3 (lower)

In [28]:
if train_segmentation == True:
    # TRAIN AND VALIDATE MODEL
    # Defect 3
    model.compile(optimizer='adam', loss=sm.losses.dice_loss,metrics=[dice_coef])
    train_batches = train_DataGenerator_3(train_data_3,shuffle=True,preprocess=preprocess)    
    valid_batches = test_DataGenerator_3(val_data_3,preprocess=preprocess)
    
    # https://www.tensorflow.org/tensorboard/r2/scalars_and_keras
    #logdir = "/content/drive/My Drive/severstal_february/severstal_logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")+'_Defect_3_01_02_2020'
    #file_writer = tf.summary.FileWriter(logdir + "/metrics")
    #tensorboard = keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=0,write_images=True)
    
    # https://keras.io/callbacks/
    mc = ModelCheckpoint('./segment_model_defect3_0105.h5', monitor='val_dice_coef', mode='max', verbose=1, save_best_only=True)
    #model training
    history = model.fit_generator(train_batches, validation_data = valid_batches, epochs = epochs, verbose=1, callbacks = [mc])

Epoch 1/30
462/462 [==============================] - 1604s 3s/step - loss: 0.4077 - dice_coef: 0.5923 - val_loss: 0.2969 - val_dice_coef: 0.5979

Epoch 00001: val_dice_coef improved from -inf to 0.59792, saving model to ./segment_model_defect3_0103.h5
Epoch 2/30
462/462 [==============================] - 579s 1s/step - loss: 0.3390 - dice_coef: 0.6610 - val_loss: 0.3147 - val_dice_coef: 0.6248

Epoch 00002: val_dice_coef improved from 0.59792 to 0.62481, saving model to ./segment_model_defect3_0103.h5
Epoch 3/30
462/462 [==============================] - 580s 1s/step - loss: 0.3264 - dice_coef: 0.6736 - val_loss: 0.2689 - val_dice_coef: 0.6596

Epoch 00003: val_dice_coef improved from 0.62481 to 0.65962, saving model to ./segment_model_defect3_0103.h5
Epoch 4/30
462/462 [==============================] - 580s 1s/step - loss: 0.3120 - dice_coef: 0.6880 - val_loss: 0.2941 - val_dice_coef: 0.6695

Epoch 00004: val_dice_coef improved from 0.65962 to 0.66955, saving model to ./segment_mode

Defect 4 

In [29]:
if train_segmentation == True:
    # TRAIN AND VALIDATE MODEL
    # Defect 4
    model.compile(optimizer='adam', loss=sm.losses.dice_loss,metrics=[dice_coef])
    train_batches = train_DataGenerator_3(train_data_4,shuffle=True,preprocess=preprocess)    
    valid_batches = test_DataGenerator_3(val_data_4,preprocess=preprocess)
    
    # https://www.tensorflow.org/tensorboard/r2/scalars_and_keras
    #logdir = "/content/drive/My Drive/severstal_february/severstal_logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")+'_Defect_4_01_02_2020'
    #file_writer = tf.summary.FileWriter(logdir + "/metrics")
    #tensorboard = keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=0,write_images=True)
    
    # https://keras.io/callbacks/
    mc = ModelCheckpoint('./segment_model_defect4_0103.h5', monitor='val_dice_coef', mode='max', verbose=1, save_best_only=True)
    #model training
    history = model.fit_generator(train_batches, validation_data = valid_batches, epochs = epochs, verbose=1, callbacks = [mc])

Epoch 1/30
72/72 [==============================] - 196s 3s/step - loss: 0.3840 - dice_coef: 0.6160 - val_loss: 0.2498 - val_dice_coef: 0.6686

Epoch 00001: val_dice_coef improved from -inf to 0.66863, saving model to ./segment_model_defect4_0103.h5
Epoch 2/30
72/72 [==============================] - 91s 1s/step - loss: 0.2766 - dice_coef: 0.7234 - val_loss: 0.1809 - val_dice_coef: 0.7321

Epoch 00002: val_dice_coef improved from 0.66863 to 0.73210, saving model to ./segment_model_defect4_0103.h5
Epoch 3/30
72/72 [==============================] - 91s 1s/step - loss: 0.2536 - dice_coef: 0.7464 - val_loss: 0.1417 - val_dice_coef: 0.7407

Epoch 00003: val_dice_coef improved from 0.73210 to 0.74068, saving model to ./segment_model_defect4_0103.h5
Epoch 4/30
72/72 [==============================] - 92s 1s/step - loss: 0.2472 - dice_coef: 0.7528 - val_loss: 0.1444 - val_dice_coef: 0.7334

Epoch 00004: val_dice_coef did not improve from 0.74068
Epoch 5/30
72/72 [=============================

# ***All Models best score***

# Binary: 
binary_model_1204.hdf5 : 0.92650

# Multi:
multi_class_model_1229.h5 : 0.89905

multi_class_model_0105.h5 : 0.90045

#Segment:
segment_model_defect1_0103.h5 : 0.68155

segment_model_defect2_0103.h5 : 0.65495

segment_model_defect2_0105.h5 : 0.65966

segment_model_defect3_0103.h5 : 0.70743

segment_model_defect4_0103.h5 : 0.77201

# ***Best performance Model set***

## Binary : binary_model_1204.hdf5

## Multi class : multi_class_model_1229.h5

## segment_defect1 : segment_model_defect1_0103.h5

## segment_defect2 : segment_model_defect2_0103.h5

## segment_defect3 : segment_model_defect3_0103.h5

## segment_defect4 : segment_model_defect4_0103.h5